In [1]:
import awswrangler as wr
import pandas as pd

### Step 1: Load All Collected Data from S3

In [2]:
s3_path = "s3://chung-yeh-youbike-poc-data/realtime/"
print("Loading data from S3... This might take a minute.")

df = wr.s3.read_parquet(path=s3_path, dataset=True)
print("Data loaded successfully!")

#Verification
print(f"DataFrame Shape: {df.shape}")
df.info()

Loading data from S3... This might take a minute.
Data loaded successfully!
DataFrame Shape: (15834106, 23)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15834106 entries, 0 to 15834105
Data columns (total 23 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   sno                     string        
 1   sna                     string        
 2   sarea                   string        
 3   mday                    string        
 4   ar                      string        
 5   sareaen                 string        
 6   snaen                   string        
 7   aren                    string        
 8   act                     string        
 9   srcUpdateTime           string        
 10  updateTime              string        
 11  infoTime                string        
 12  infoDate                string        
 13  total                   Int64         
 14  available_rent_bikes    Int64         
 15  latitude                

### Step 2: Data Cleaning & Type Conversion

In [3]:
# 1. Convert API timestamp columns to proper datetime objects
df['mday'] = pd.to_datetime(df['mday'])
df['updateTime'] = pd.to_datetime(df['updateTime'])
df['infoTime'] = pd.to_datetime(df['infoTime'])

In [6]:
# 2. Drop unnecessary or redundant columns
columns_to_drop = ['srcUpdateTime', 'infoDate', 'year', 'month', 'day',"Quantity"]
df = df.drop(columns=columns_to_drop)
print("Dropped unnecessary columns.")

Dropped unnecessary columns.


In [7]:
# 3. Handle Duplicates
print(f"Original shape: {df.shape}")
df = df.drop_duplicates(subset=['sno', 'collection_timestamp'], keep='last')
print(f"Shape after dropping duplicates: {df.shape}")

Original shape: (15834106, 17)
Shape after dropping duplicates: (15823996, 17)


In [8]:
# 4. Set a Multi-Index
df = df.set_index(['sno', 'collection_timestamp']).sort_index()
print("Set and sorted a multi-index.")

Set and sorted a multi-index.


In [9]:
# Verification
print("\nCleaned DataFrame Info:")
df.info()
print("\nCleaned DataFrame Head:")
print(df.head())


Cleaned DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15823996 entries, ('500101001', Timestamp('2025-06-18 15:03:19.766456')) to ('500119097', Timestamp('2025-07-22 16:33:15.789889'))
Data columns (total 15 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   sna                     string        
 1   sarea                   string        
 2   mday                    datetime64[ns]
 3   ar                      string        
 4   sareaen                 string        
 5   snaen                   string        
 6   aren                    string        
 7   act                     string        
 8   updateTime              datetime64[ns]
 9   infoTime                datetime64[ns]
 10  total                   Int64         
 11  available_rent_bikes    Int64         
 12  latitude                float64       
 13  longitude               float64       
 14  available_return_bikes  Int64         
dtypes: Int64

### Step 4: Feature Engineering

In [11]:
# 1. Create Time-Based Features
print("Creating time-based features...")
# We reset the index to access 'collection_timestamp' as a column
df.reset_index(inplace=True)
df['hour'] = df['collection_timestamp'].dt.hour
df['day_of_week'] = df['collection_timestamp'].dt.dayofweek # Monday=0, Sunday=6
df['is_weekend'] = (df['day_of_week'] >= 5).astype(int)
df.set_index(['sno', 'collection_timestamp'], inplace=True) # Set the index back
print(df[['hour', 'day_of_week', 'is_weekend']].head())

Creating time-based features...
                                      hour  day_of_week  is_weekend
sno       collection_timestamp                                     
500101001 2025-06-18 15:03:19.766456    15            2           0
          2025-06-18 15:08:15.925407    15            2           0
          2025-06-18 15:13:15.744279    15            2           0
          2025-06-18 15:18:15.684246    15            2           0
          2025-06-18 15:23:15.863098    15            2           0


In [12]:
# 2. Create the Target Variable
print("\nCreating the target variable 'status_in_15_mins'...")

# Predict the state 15 minutes from now (3 steps in the future)
SHIFT_PERIOD = -3

# We must sort the data before shifting to ensure we get the correct future value.
df.sort_values(['sno', 'collection_timestamp'], inplace=True)

# Ensure we only look at the future of the same station(3 rows in the future)
future_bikes = df.groupby('sno')['available_rent_bikes'].shift(SHIFT_PERIOD)
future_docks = df.groupby('sno')['available_return_bikes'].shift(SHIFT_PERIOD)

# Define the conditions for our classification target
# Handle NaN values at the end of each group, preventing the TypeError.
conditions = [
    (future_bikes == 0).fillna(False),
    (future_docks == 0).fillna(False),
    (future_bikes <= 3).fillna(False)
]

# Define the outcomes for each condition
outcomes = ['EMPTY', 'FULL', 'LOW']

# Create the target column.
# If none of the conditions are met, the default value is 'HEALTHY'.
import numpy as np
df['status_in_15_mins'] = np.select(conditions, outcomes, default='HEALTHY')

# Drop the rows where the target is NaN
df.dropna(subset=['status_in_15_mins'], inplace=True)

print("\nTarget variable created. Value counts:")
print(df['status_in_15_mins'].value_counts(normalize=True))


Creating the target variable 'status_in_15_mins'...

Target variable created. Value counts:
status_in_15_mins
HEALTHY    0.752322
LOW        0.172733
EMPTY      0.054585
FULL       0.020360
Name: proportion, dtype: float64


In [13]:
# 3. Save Final Dataset for Modeling
final_s3_path = "s3://chung-yeh-youbike-poc-data/processed/youbike_data_for_modeling.parquet"
print(f"\nSaving final dataset to {final_s3_path}...")
wr.s3.to_parquet(df=df, path=final_s3_path, index=True)
print("Final dataset saved successfully!")


Saving final dataset to s3://chung-yeh-youbike-poc-data/processed/youbike_data_for_modeling.parquet...
Final dataset saved successfully!
